# 2. Labelling the data by Sentiment and Emotion

Downloading the necessary packages so far for labelling the data

In [ ]:
# downloading the relevant modules
!pip install transformers torch numpy pandas tqdm


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive', force_remount=True) # Adding force_remount=True to force the remounting process

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/data_universities_only.csv'


In [ ]:
import pandas as pd
df_reviews = pd.read_csv(file_path)
df_reviews

,reviews,date_reviews,rating,course_id,month,departments,name,institution
0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
...,...,...,...,...,...,...,...,...
58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan
58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan


**Manually Labelling by sentiment based on the rating **

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, pipeline
import numpy as np
import torch
import os
from tqdm import tqdm

# Define a function to label sentiment based on numerical ratings
def label_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'

# Apply the function to create a sentiment label based on ratings
df_reviews['sentiment_from_rating'] = df_reviews['rating'].apply(label_sentiment)
df_reviews

,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating
0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative
1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative
2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative
3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative
4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative
...,...,...,...,...,...,...,...,...,...
58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive
58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive
58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive
58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive


**Need to figure out how to label in parallel to complete the basic labelling in one go, otherwise it's gnna take extra long
**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = AutoModelForSequenceClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, pipeline
from tqdm import tqdm
import torch
import os

# Load model and tokenizer
model_name = "monologg/bert-base-cased-goemotions-original"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the emotion classifier pipeline with truncation enabled
emotion_classifier = pipeline(
    "text-classification",
    model=model_name,
    tokenizer=tokenizer,
    truncation=True,       # Automatically truncate inputs longer than 512 tokens
    max_length=512         # Set the max length to 512 tokens
)

# Define constants
BATCH_SIZE = 16
CHECKPOINT_DIR = '/content/drive/MyDrive/checkpoints30'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_PTH_PATH = os.path.join(CHECKPOINT_DIR, 'emotion_predictions.pth')

# Initialize list for predictions
all_predictions = []

# Checkpoint functions
def save_checkpoint(all_predictions, filepath):
    torch.save(all_predictions, filepath)

def load_checkpoint(filepath):
    return torch.load(filepath)

# Load checkpoint if available
if os.path.exists(CHECKPOINT_PTH_PATH):
    all_predictions = load_checkpoint(CHECKPOINT_PTH_PATH)
    start_index = len(all_predictions)
else:
    start_index = 0

# Process each review with checkpointing
for i in tqdm(range(start_index, len(df_reviews), BATCH_SIZE)):
    batch_reviews = df_reviews['reviews'].iloc[i:i + BATCH_SIZE]
    batch_predictions = []

    # Process each review in the batch
    for review in batch_reviews:
        # Check token length (including special tokens)
        token_length = len(tokenizer(review)['input_ids'])

        # Debugging: Print the review and its token length
        # print(f"Review: {review[:50]}... | Token Length: {token_length}")  # Print first 50 characters and token length

        if token_length > 512:
            # Skip reviews that exceed 512 tokens
            batch_predictions.append(None)  # Append None for skipped reviews
            continue  # Skip the rest of the loop for this review
        else:
          # Classify the review (this uses the pipeline which handles truncation internally)
          try:
              prediction = emotion_classifier(review)
              emotion = max(prediction, key=lambda x: x['score'])['label']
              # Add the aggregated emotion label for this review
              batch_predictions.append(emotion)
          except Exception as e:
              print(f"Error processing review: {review[:50]}... | Error: {str(e)}")  # Log any errors encountered
              batch_predictions.append(None)  # Append None for errors

    # Append batch predictions to all predictions, filtering out None values
    all_predictions.extend(batch_predictions)

    # Save checkpoint after processing each batch
    save_checkpoint(all_predictions, CHECKPOINT_PTH_PATH)

    # Update the emotion_label column with predictions, ensuring we do not overwrite skipped reviews
    for idx, pred in zip(range(start_index, start_index + len(batch_predictions)), batch_predictions):
        if pred is not None:  # Only assign emotion if it was classified
            df_reviews.at[idx, 'emotion_label'] = pred

    # Save checkpoint CSV after processing each batch
    # not efficient for space, so going to stop saving the csvs and delete previous ones since I now know it works
    # csv_checkpoint_path = os.path.join(CHECKPOINT_DIR, f'checkpoint_batch_{i + BATCH_SIZE}.csv')
    # df_reviews.to_csv(csv_checkpoint_path, index=False)

    # Update start_index for the next batch
    start_index += len(batch_predictions)

# Final assignment of predictions to DataFrame
df_reviews['emotion_label'] = all_predictions
print(df_reviews[['reviews', 'emotion_label']].head())


Output hidden; open in https://colab.research.google.com to view.

Testing the parallel labelling

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, pipeline
from tqdm import tqdm
import torch
import os

df_reviews_5 = df_reviews.head(5)

# Load the main emotion model and tokenizer
emotion_model_name = "monologg/bert-base-cased-goemotions-original"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)
emotion_classifier = pipeline(
    "text-classification",
    model=emotion_model_name,
    tokenizer=emotion_tokenizer,
    truncation=True,
    max_length=512
)

# Load the sentiment model and tokenizer
sentiment_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_classifier = pipeline(
    "text-classification",
    model=sentiment_model_name,
    tokenizer=sentiment_tokenizer,
    truncation=True,
    max_length=512
)

# Define directories and checkpoints
BATCH_SIZE = 5
CHECKPOINT_DIR = '/content/drive/MyDrive/checkpoints30'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_PTH_PATH = os.path.join(CHECKPOINT_DIR, 'emotion_sentiment_predictions.pth')

# Initialize list for predictions
all_emotion_predictions = []
all_sentiment_predictions = []

# Checkpoint functions
def save_checkpoint(emotion_predictions, sentiment_predictions, filepath):
    torch.save({'emotion': emotion_predictions, 'sentiment': sentiment_predictions}, filepath)

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    return checkpoint['emotion'], checkpoint['sentiment']

# Load checkpoint if available
if os.path.exists(CHECKPOINT_PTH_PATH):
    all_emotion_predictions, all_sentiment_predictions = load_checkpoint(CHECKPOINT_PTH_PATH)
    start_index = len(all_emotion_predictions)
else:
    start_index = 0

# Process each review with checkpointing
for i in tqdm(range(start_index, len(df_reviews_5), BATCH_SIZE)):
    batch_reviews = df_reviews_5['reviews'].iloc[i:i + BATCH_SIZE]
    batch_emotion_predictions = []
    batch_sentiment_predictions = []

    # Process each review in the batch
    for review in batch_reviews:
        # Check token length for the emotion model
        token_length = len(emotion_tokenizer(review)['input_ids'])
        if token_length > 512:
            batch_emotion_predictions.append(None)
            batch_sentiment_predictions.append(None)
            continue  # Skip reviews that exceed 512 tokens

        # Emotion Classification (with truncation handled internally)
        try:
            emotion_prediction = emotion_classifier(review)
            emotion = max(emotion_prediction, key=lambda x: x['score'])['label']
            batch_emotion_predictions.append(emotion)
        except Exception as e:
            print(f"Error processing emotion for review: {review[:50]}... | Error: {str(e)}")
            batch_emotion_predictions.append(None)

        # Sentiment Classification (with truncation handled internally)
        try:
            sentiment_prediction = sentiment_classifier(review)
            sentiment = max(sentiment_prediction, key=lambda x: x['score'])['label']
            batch_sentiment_predictions.append(sentiment)
        except Exception as e:
            print(f"Error processing sentiment for review: {review[:50]}... | Error: {str(e)}")
            batch_sentiment_predictions.append(None)

    # Append batch predictions to all predictions
    all_emotion_predictions.extend(batch_emotion_predictions)
    all_sentiment_predictions.extend(batch_sentiment_predictions)

    # Save checkpoint after processing each batch
    save_checkpoint(all_emotion_predictions, all_sentiment_predictions, CHECKPOINT_PTH_PATH)

    # Update the emotion_label and sentiment_label columns with predictions
    for idx, (emotion_pred, sentiment_pred) in enumerate(zip(batch_emotion_predictions, batch_sentiment_predictions)):
        df_reviews_5.at[start_index + idx, 'emotion_label'] = emotion_pred
        df_reviews_5.at[start_index + idx, 'sentiment_label'] = sentiment_pred

    # Update start_index for the next batch
    start_index += len(batch_emotion_predictions)

# Map the star ratings to sentiments
def map_rating_to_sentiment(rating):
    if rating in ['1 star', '2 stars']:
        return 'Negative'
    elif rating == '3 stars':
        return 'Neutral'
    elif rating in ['4 stars', '5 stars']:
        return 'Positive'
    else:
        return 'Unknown'  # For any unexpected values

# Apply the mapping function to create a new column 'mapped_sentiment' based on 'sentiment_label'
df_reviews_5['mapped_sentiment'] = df_reviews_5['sentiment_label'].apply(map_rating_to_sentiment)

# Display the final DataFrame with both emotion and mapped sentiment columns
print(df_reviews_5[['reviews', 'emotion_label', 'sentiment_label', 'mapped_sentiment']].head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-5-19cd05998d67>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_5.at[start_index + idx, 'emotion_label'] = emotion_pred
<ipython-input-5-19cd05998d67>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_5.at[start_index + idx, 'sentiment_label'] = sentiment_pred
100%|██████████| 1/1 [00:04<00:00,  4.29s/it]

                                             reviews emotion_label  \
0  I did not find this course helpful. The assign...   disapproval   
1  I got a 95% and maybe I learned something but ...       neutral   
2  Two weeks of all sort of nonsense, including m...      surprise   
3  We tried to use this class to teach a brown ba...    admiration   
4  To be honest, I thought the course stumbled an...     confusion   

  sentiment_label mapped_sentiment  
0         2 stars         Negative  
1         2 stars         Negative  
2          1 star         Negative  
3         2 stars         Negative  
4         2 stars         Negative  



<ipython-input-5-19cd05998d67>:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_5['mapped_sentiment'] = df_reviews_5['sentiment_label'].apply(map_rating_to_sentiment)


parallel labelling the whole dataset

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, pipeline
from tqdm import tqdm
import torch
import os

# Load models and tokenizers
emotion_model_name = "monologg/bert-base-cased-goemotions-original"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)
emotion_classifier = pipeline(
    "text-classification",
    model=emotion_model_name,
    tokenizer=emotion_tokenizer,
    truncation=True,
    max_length=512
)

sentiment_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_classifier = pipeline(
    "text-classification",
    model=sentiment_model_name,
    tokenizer=sentiment_tokenizer,
    truncation=True,
    max_length=512
)

# Define checkpoint path
CHECKPOINT_DIR = '/content/drive/MyDrive/checkpoints35'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, 'predictions_checkpoint.pth')

# Initialize lists for predictions
all_emotion_predictions = []
all_sentiment_predictions = []

# Check if a checkpoint exists to load saved progress
if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH)
    all_emotion_predictions = checkpoint['emotion_predictions']
    all_sentiment_predictions = checkpoint['sentiment_predictions']
    start_index = len(all_emotion_predictions)  # Resume from the next batch
else:
    start_index = 0  # Start from the beginning if no checkpoint exists

# Loop through reviews in batches
BATCH_SIZE = 16  # Set batch size to save checkpoints after every 16 reviews
for i in tqdm(range(start_index, len(df_reviews), BATCH_SIZE)):
    batch_reviews = df_reviews['reviews'].iloc[i:i + BATCH_SIZE]

    # Initialize lists for batch predictions
    batch_emotion_predictions = []
    batch_sentiment_predictions = []

    for review in batch_reviews:
        # Tokenize with truncation applied
        emotion_input = emotion_tokenizer(review, truncation=True, max_length=512, return_tensors="pt")
        sentiment_input = sentiment_tokenizer(review, truncation=True, max_length=512, return_tensors="pt")

        if len(emotion_input['input_ids'][0]) > 512 or len(sentiment_input['input_ids'][0]) > 512:
            # If tokenized length exceeds 512, skip this review
            batch_emotion_predictions.append(None)
            batch_sentiment_predictions.append(None)
            continue

        try:
            # Process emotion prediction
            emotion_prediction = emotion_classifier(review)
            emotion = max(emotion_prediction, key=lambda x: x['score'])['label']
            batch_emotion_predictions.append(emotion)

            # Process sentiment prediction
            sentiment_prediction = sentiment_classifier(review)
            sentiment = max(sentiment_prediction, key=lambda x: x['score'])['label']
            batch_sentiment_predictions.append(sentiment)

        except Exception as e:
            print(f"Error processing review: {review[:50]}... | Error: {str(e)}")
            batch_emotion_predictions.append(None)
            batch_sentiment_predictions.append(None)

    # Append batch predictions to main lists
    all_emotion_predictions.extend(batch_emotion_predictions)
    all_sentiment_predictions.extend(batch_sentiment_predictions)

    # Save checkpoint after each batch
    torch.save({
        'emotion_predictions': all_emotion_predictions,
        'sentiment_predictions': all_sentiment_predictions
    }, CHECKPOINT_PATH)

# Assign predictions to DataFrame columns
df_reviews['emotion_label'] = all_emotion_predictions
df_reviews['sentiment_label'] = all_sentiment_predictions

# Display the DataFrame with new columns
print(df_reviews[['reviews', 'emotion_label', 'sentiment_label']].head())


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-5-c7992b6ad1db>:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

                                             reviews emotion_label  \
0  I did not find this course helpful. The assign...   disapproval   
1  I got a 95% and maybe I learned something but ...       neutral   
2  Two weeks of all sort of nonsense, including m...      surprise   
3  We tried to use this class to teach a brown ba...    admiration   
4  To be honest, I thought the course stumbled an...     confusion   

  sentiment_label  
0         2 stars  
1         2 stars  
2          1 star  
3         2 stars  
4         2 stars  


After the dataset is labelled, need to delete reviews that would have exceeded the 512 token limit (there's about 100) and also remove reviews that are not in english -> can probably use another model to do that. Then I need to save as a csv dataset and maybe remove the unlabelled one. And also remove neutral emotion / sentiment labels. Need to get performance as well by comparison but not crazy brazy pivotal.

In [ ]:
df_reviews

,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label
0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars
1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars
2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star
3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars
4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars
...,...,...,...,...,...,...,...,...,...,...,...
58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars
58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars
58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars
58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars


In [ ]:
df_reviews.to_csv('/content/drive/MyDrive/labelled_data.csv')

map review number to sentiment

In [ ]:
import pandas as pd
df_reviews_labelled = pd.read_csv('/content/drive/MyDrive/labelled_data.csv')
df_reviews_labelled

,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label
0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars
1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars
2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star
3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars
4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars
...,...,...,...,...,...,...,...,...,...,...,...,...
58724,58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars
58725,58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars
58726,58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars
58727,58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars


In [ ]:
# Define a function to map star ratings to sentiments
def map_rating_to_sentiment(rating):
    # Extract the numeric part from the 'stars' string
    try:
        stars = int(rating.split()[0])
        if stars > 3:
            return 'positive'
        elif stars < 3:
            return 'negative'
        else:
            return 'neutral'
    except (ValueError, IndexError):
        return 'Unknown'  # Handle cases where format might be unexpected

# Apply the function to the 'sentiment_label' column
df_reviews_labelled['sentiment_label_from_review'] = df_reviews_labelled['sentiment_label'].apply(map_rating_to_sentiment)

# Display the DataFrame with the new sentiment_mapped column
df_reviews_labelled

,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative
2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative
3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58724,58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
58725,58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars,positive
58726,58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
58727,58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars,positive


deleting non-english reviews

In [ ]:
! pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=18c080cfa5420b21fdc5abb842f99293f49a46dfa88915e646125e7440f869c1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect, DetectorFactory
import pandas as pd

# Fix seed to ensure reproducibility in langdetect
DetectorFactory.seed = 0

# Function to check if a review is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False  # If detection fails, assume it's not English

# Sample DataFrame for context (replace with your actual data)
# df_reviews_labelled = pd.DataFrame({'reviews': ["This is a great course!", "Este es un gran curso!", "C'est un excellent cours!"]})

# Apply the function and filter the DataFrame
df_reviews_labelled['is_english'] = df_reviews_labelled['reviews'].apply(is_english)

# Filter out non-English reviews
df_reviews_english = df_reviews_labelled[df_reviews_labelled['is_english']]

# Drop the 'is_english' column if no longer needed
df_reviews_english = df_reviews_english.drop(columns=['is_english'])

# Display the filtered DataFrame
df_reviews_english


,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative
2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative
3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58724,58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
58725,58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars,positive
58726,58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
58727,58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars,positive


deleting reviews w token size < 512

In [ ]:
from transformers import AutoTokenizer
import pandas as pd

# Initialize the tokenizer
sentiment_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)

# Sample DataFrame for context (replace with your actual data)
# df_reviews = pd.DataFrame({'reviews': ["This is a sample review...", "Another review that is much longer...", "..."]})

# Function to check if token count is within limit
def is_within_token_limit(review, max_length=512):
    # Tokenize and check if token count is within the limit
    token_length = len(sentiment_tokenizer.encode(review, add_special_tokens=True))
    return token_length <= max_length

# Apply the function and filter the DataFrame
df_reviews_english = df_reviews_english[df_reviews_english['reviews'].apply(is_within_token_limit)].reset_index(drop=True)

# Display the filtered DataFrame
df_reviews_english


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (531 > 512). Running this sequence through the model will result in indexing errors


,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative
2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative
3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56160,58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
56161,58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars,positive
56162,58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
56163,58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars,positive


deleting reviews with neutral label in emotion or sentiment

In [ ]:
# Filter the DataFrame to exclude rows where either column has a 'Neutral' rating
df_reviews_english = df_reviews_english[
    (df_reviews_english['sentiment_from_rating'] != 'neutral') &
    (df_reviews_english['sentiment_label_from_review'] != 'neutral')
].reset_index(drop=True)

# Display the filtered DataFrame
df_reviews_english


,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative
2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative
3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49668,58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
49669,58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars,positive
49670,58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
49671,58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars,positive


In [ ]:
df_reviews_english.to_csv('/content/drive/MyDrive/labelled_data_further_processed.csv')